In [1]:
using PyPlot
using Optim

In [2]:
# Convert an Float32 rbg image to Float64 grayscale image
function convert_to_grayscale(I::Array{Float32,3})
    I=convert(Array{Float64,3}, I)
    I_gray = 0.2989*I[:,:,1] + 0.5870*I[:,:,2] + 0.1140*I[:,:,3]
    return I_gray::Array{Float64,2}
end

convert_to_grayscale (generic function with 1 method)

In [3]:
# Load Tsukuba disparity dataset and convert it to grayscale
function load_data()
    i0_path = string(@__DIR__,"/i0.png")
    i0 = imread(i0_path)
    i0 = convert_to_grayscale(i0)
    i1_path = string(@__DIR__,"/i1.png")
    i1 = imread(i1_path)
    i1 = convert_to_grayscale(i1)
    gt_path = string(@__DIR__,"/gt.png")
    gt64 = convert(Array{Float64,2}, imread(gt_path)*255)

    @assert maximum(gt64) <= 16
    return i0::Array{Float64,2}, i1::Array{Float64,2}, gt64::Array{Float64,2}
end

load_data (generic function with 1 method)

In [16]:
# Robust loss function returning function value and gradient
function robust_func(x::Float64,alpha::Float64,c::Float64)
    if alpha == 2.0
        value = 0.5*(x/c)^2
        gradient = x/(c^2)
    elseif alpha == 0.0
        value = log(0.5*(x/c)^2 + 1.0)
        gradient = (2*x)/(x^2 + 2*c^2)
    # TODO: case alpha == -inf ?
    else
        value = abs(alpha-2.0)/alpha * (((x/c)^2/abs(alpha-2) + 1.0)^(alpha/2) - 1.0)
        gradient = x/(c^2) * ((x/c)^2/abs(alpha-2.0) + 1.0)^(alpha/2 - 1)
    end
    # print("\nValue of robust func: ", value)
    return value::Float64, gradient::Float64
end

robust_func (generic function with 1 method)

In [17]:
function prior(x::Array{Float64,2})
    # TODO: Energy formularisation; Use sum instead of product!
    alpha = 2.0
    c = 1.0
    prior = 0
    grad_prior = zeros(size(x))
    for i = 1:size(x)[1]-1
        for j = 1:size(x)[2]-1
            d1 = x[i,j]-x[i+1,j]
            d2 = x[i,j]-x[i,j+1]
            prior += robust_func(d1, alpha, c)[1] + robust_func(d2, alpha, c)[1]
            grad = 0
            if i+1 <= size(x)[1]
                grad += robust_func(x[i,j]-x[i+1,j], alpha, c)[2]
            end
            if i-1 >= 1
                grad += robust_func(x[i-1,j]-x[i,j], alpha, c)[2]
            end
            if j+1 <= size(x)[2]
                grad += robust_func(x[i,j]-x[i,j+1], alpha, c)[2]
            end
            if j-1 >= 1
                grad += robust_func(x[i,j-1]-x[i,j], alpha, c)[2]
            end
            grad_prior[i,j] = grad
        end
    end
    # TODO: Normalization
    return prior, grad_prior
end

prior (generic function with 1 method)

In [18]:
# Calculates the gradient of a given image in both directions
function image_gradient(im::Array{Float64,2})
    horizontal_im_grad = zeros(size(im))
    vertical_im_grad = zeros(size(im))
    for x = 1:size(im)[1]
        for y = 1:size(im)[2]
            if x-1 > 0
                vertical_im_grad[x,y] -= im[x-1,y]
            end
            if x+1 < size(im)[1]
                vertical_im_grad[x,y] += im[x+1,y]
            end
            if y-1 > 0
                horizontal_im_grad[x,y] -= im[x,y-1]
            end
            if y+1 < size(im)[2]
                horizontal_im_grad[x,y] += im[x,y+1]
            end
        end
    end
    return horizontal_im_grad::Array{Float64,2}, vertical_im_grad::Array{Float64,2}
end

image_gradient (generic function with 1 method)

In [19]:
# Shift all pixels of i1 to the right by the value of gt
function shift_disparity(i1::Array{Float64,2}, gt::Array{Float64,2})
    # TODO: do interpolation... constant disparity map
    @assert size(i1) == size(gt)
    id = zeros(size(i1))
    for x = 1:size(i1)[2] 
        for y = 1:size(i1)[1]
            # shifted_index = Int64(floor(x + gt[y,x]))
            shifted_index = Int64(floor(clamp(x + gt[y,x],0.0, 255.0)))
            if (shifted_index > 1) && (shifted_index < size(i1,2))
                id[y, shifted_index] = i1[y,x]          
            end
        end
    end
    @assert size(id) == size(i1)
    return id::Array{Float64,2}
end

shift_disparity (generic function with 1 method)

In [20]:
function likelihood(x::Array{Float64, 2}, im0::Array{Float64, 2}, im1::Array{Float64, 2})
    alpha = 0.0
    c = 1.0
    lh = 0
    lh_grad = zeros(size(im1))
    # We can shift I1 by the disparity first
    im1_d = shift_disparity(im1, x)
    # We need the horizontal image derivative from I1 to calculate the gradient of the LH
    h_img_grad = image_gradient(im1_d)[1]
    for i = 1:size(x)[1]
        for j = 1:size(x)[2]
            d = im0[i,j]-im1_d[i, j]
            lh += robust_func(d, alpha, c)[1]
            lh_grad[i,j] = (-1)*robust_func(d, alpha, c)[2]*h_img_grad[i, j]         
        end
    end
    return lh::Float64, lh_grad::Array{Float64,2}
end

likelihood (generic function with 1 method)

In [21]:
function posterior(x::Array{Float64, 2}, im0::Array{Float64, 2}, im1::Array{Float64, 2})
    # (We can again drop the marginalisation terms)
    post = likelihood(x, im0, im1)[1] + prior(x)[1] + prior(im0)[1]
    grad_lh = likelihood(x, im0, im1)[2]
    grad_x = prior(x)[2]
    # (Derivative of I0 to x should be zero, so we drop it ...)
    post_grad = grad_lh + grad_x
    return post::Float64, post_grad::Array{Float64,2}
end

posterior (generic function with 1 method)

In [30]:
function stereo(x0::Array{Float64, 2}, im0::Array{Float64, 2}, im1::Array{Float64, 2})
    println("\nRunning Stereo Algorithm...")
    # Helper function with fixed im0 and im1
    function f(y)
        value = -posterior(y, im0, im1)[1]
        return value
    end
    function g!(storage, y)
        grad = -posterior(y, im0, im1)[2]
        storage[:,:] = grad
    end
    options = Optim.Options(iterations=200, show_trace=true, allow_f_increases=true)
    # Specify optim algorithm here
    res = optimize(f, g!, x0, ConjugateGradient(), options)
    x = Optim.maximizer(res)
    print("done!")
    return x::Array{Float64,2}
end

stereo (generic function with 1 method)

In [23]:
# create constant disparity of all 8's of size DISPARITY_SIZE
function constant_disparity(disparity_size::Tuple{Int64,Int64})
    disparity_map = fill(8.0, disparity_size)
    return disparity_map::Array{Float64,2}
end

constant_disparity (generic function with 1 method)

In [24]:
# Create random disparity in [0,14] of size DISPARITY_SIZE
# We changed DISPARITY_SIZE to a tuple of integers
function random_disparity(disparity_size::Tuple{Int64,Int64})
    disparity_map = Array{Float64,2}(rand(collect(1:14),disparity_size))
    return disparity_map::Array{Float64,2}
end

random_disparity (generic function with 1 method)

In [25]:
function problem4()
    #  Up to you...
    im0, im1, gt = load_data()
    #print("Prior GT: ", prior(gt)[1])
    
     # Display stereo: Initialized with constant 8's
    const_d = constant_disparity(size(gt))
    #print("\nPrior constant disparity: ", prior(const_d)[1])
    
    # Display stereo: Initialized with noise in [0,14]
    random_d = random_disparity(size(gt))
    #print("\nPrior random disparity: ", prior(random_d)[1])
    
    # print("\nLH const disp: ", stereo_log_likelihood(const_d, im0, im1)[1])
    x = stereo(const_d, im0, im1)
    imshow(x)
    
    # TODO: pyramid
end

problem4 (generic function with 1 method)

In [31]:
problem4()


Running Stereo Algorithm...
Iter     Function value   Gradient norm 
     0    -2.309055e+03     3.874187e-01
     1    -4.329334e+06     2.686511e+02
     2    -1.838070e+08     1.752539e+03
     3    -7.934616e+08     3.641969e+03
     4    -3.328513e+09     7.460678e+03
     5    -1.334852e+10     1.494327e+04
     6    -5.323462e+10     2.984705e+04
     7    -2.136159e+11     5.979944e+04
     8    -8.541591e+11     1.195983e+05
     9    -3.417521e+12     2.392691e+05
    10    -1.367480e+13     4.787039e+05
    11    -5.467421e+13     9.573549e+05
    12    -2.187295e+14     1.915185e+06
    13    -8.748056e+14     3.830786e+06
    14    -3.499130e+15     7.662796e+06
    15    -1.399593e+16     1.532791e+07
    16    -5.598080e+16     3.066032e+07
    17    -2.239132e+17     6.132985e+07
    18    -8.956122e+17     1.226780e+08
    19    -3.582285e+18     2.453926e+08
    20    -1.432848e+19     4.908583e+08
    21    -5.731131e+19     9.818628e+08
    22    -2.292348e+20     

   199   -8.354003e+126     3.854109e+62
   200   -3.341493e+127     7.709172e+62


MethodError: MethodError: no method matching MethodError()
Closest candidates are:
  MethodError(!Matched::Any, !Matched::Any) at boot.jl:298
  MethodError(!Matched::Any, !Matched::Any, !Matched::UInt64) at boot.jl:295

UndefVarError: UndefVarError: gt not defined